In [ ]:
import os
os.chdir('/mnt/c/Users/ralvin/OneDrive - Reliant Health Partners/Documents/RHP_dev_RA/Automate_skyvia')
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin
import pyodbc, sys, time 
import requests
import numpy as np
from datetime import datetime
from helper_functions_v2 import create_df, updated_data_pull, upsert, delete_record, insert_records, salesforce_connection, read_sftp_data
from openai import OpenAI
import ast

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

sf = salesforce_connection(sandbox=True)
sf_prod = salesforce_connection(sandbox=False)


provider_data_test = updated_data_pull(['Provider_TIN__c'], sf)
provider_data_real = updated_data_pull(['Provider_TIN__c'], sf_prod)


API_key = "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"
temp = provider_data_real.loc[:, ['Special_Repricing__c', 'Notes__c']]
temp[temp['Special_Repricing__c']!=False].drop_duplicates(subset=['Notes__c']).head(10)

list_of_notes = list(temp[temp['Special_Repricing__c']!=False].drop_duplicates(subset=['Notes__c'])['Notes__c'])
print(len(list_of_notes))



Connected to Salesforce sandbox
Connected to Salesforce Prod
pulled Provider_TIN__c
pulled Provider_TIN__c
713


In [16]:
list_of_notes[:10]


['8/23/24 - bk - Non Par for QON/Broadspire',
 'appealed pmt - 100% went to deduct',
 'process 100%',
 '6/18/24 - bk - Please red flag for Novanet.',
 'appeals CT work for FS after 4/1/2015',
 'appeals CT work comp',
 'appealed wc',
 'appeals CT work comp below FS',
 'provider balance bills, process 100%',
 'process in full']

In [7]:
client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = API_key
)

def LLM(prompt):


  completion = client.chat.completions.create(
  model="meta/llama-3.3-70b-instruct",
  messages=prompt['messages'],
  temperature=0.2,
  top_p=0.7,
  max_tokens=5000,
  stream=True
)
  

  output = ""
  for chunk in completion:
    if chunk.choices[0].delta.content is not None:
      # print(chunk.choices[0].delta.content, end="")
      output += chunk.choices[0].delta.content

  return output



prompt1 = f"""
Analyze a list of strings containing notes about a specific insurance provider.
Group the strings into meaningful categories based on their content, without predefining the categories.
Instead, derive the categories dynamically from the data. 
The goal is to standardize the strings by organizing them into these content-driven categories. 
This here is the list of stings: {list_of_notes[:]}. Generate only the final list of distinct category labels identified during the analysis. 
Do not generate the steps needed to compose the final list of distinct categories
"""

prompt2 = f"""

Now that we have defined a list of distinct category labels, analyze each string in the provided input list and assign an appropriate class label from the predefined categories. 
The output should be a list of class labels, where each label corresponds to a string in the input list.'
Ensure the output list is the same length as the input list. This here is the list of stings: {list_of_notes[:]} and this is the length of the list: {len(list_of_notes[:])}.
Generate only the final list of assigned class labels, without including intermediate steps or explanations.

"""


prompt = {
    "messages": [
        {
            "role": "system",
            "content": prompt1
        },
        {
            "role": "user",
            "content": prompt2
        }
    ]
}

output = LLM(prompt)



In [15]:
print(output)


['Non Par', 'Appeals', 'Process 100%', 'Non Par', 'Appeals', 'Appeals', 'Appeals', 'Appeals', 'Balance Bill', 'Process 100%', 'Process 100%', 'Non Par', 'Rate Update', 'Non Par', 'Non Par', 'Standard Discount', 'Process 100%', 'Process 100%', 'Discount Request', 'Process 100%', 'Appeals', 'Appeals', 'Non Par', 'Non Par', 'Non Par', 'Increase Allowed Amounts', 'Rate Information', 'Appeals', 'Rate Information', 'Balance Bill', None, 'Non Par', 'Discount Request', 'Balance Bill', 'Appeals', 'Non Par', 'Non Par', 'Balance Bill', 'Appeals', 'Appeals', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Appeals', 'Appeals', 'Appeals', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Pa

In [10]:
trimmed_string = output.rsplit(',', 1)[0] + "]"

# Step 2: Parse into a list
try:
    result_list = ast.literal_eval(trimmed_string)
    print("Parsed List:", result_list)
except (SyntaxError, ValueError) as e:
    print("Error parsing the string:", e)

Parsed List: ['Non Par', 'Appeals', 'Process 100%', 'Non Par', 'Appeals', 'Appeals', 'Appeals', 'Appeals', 'Balance Bill', 'Process 100%', 'Process 100%', 'Non Par', 'Rate Update', 'Non Par', 'Non Par', 'Standard Discount', 'Process 100%', 'Process 100%', 'Discount Request', 'Process 100%', 'Appeals', 'Appeals', 'Non Par', 'Non Par', 'Non Par', 'Increase Allowed Amounts', 'Rate Information', 'Appeals', 'Rate Information', 'Balance Bill', None, 'Non Par', 'Discount Request', 'Balance Bill', 'Appeals', 'Non Par', 'Non Par', 'Balance Bill', 'Appeals', 'Appeals', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Appeals', 'Appeals', 'Appeals', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non Par', 'Non 

In [13]:
result_list

['Non Par',
 'Appeals',
 'Process 100%',
 'Non Par',
 'Appeals',
 'Appeals',
 'Appeals',
 'Appeals',
 'Balance Bill',
 'Process 100%',
 'Process 100%',
 'Non Par',
 'Rate Update',
 'Non Par',
 'Non Par',
 'Standard Discount',
 'Process 100%',
 'Process 100%',
 'Discount Request',
 'Process 100%',
 'Appeals',
 'Appeals',
 'Non Par',
 'Non Par',
 'Non Par',
 'Increase Allowed Amounts',
 'Rate Information',
 'Appeals',
 'Rate Information',
 'Balance Bill',
 None,
 'Non Par',
 'Discount Request',
 'Balance Bill',
 'Appeals',
 'Non Par',
 'Non Par',
 'Balance Bill',
 'Appeals',
 'Appeals',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Appeals',
 'Appeals',
 'Appeals',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',
 'Non Par',


In [27]:
eval(output)

['Non-Par Provider',
 'Payment Appeal',
 'Full Payment',
 'Red Flag',
 'Work Comp Appeal',
 'Work Comp Appeal',
 'Work Comp Appeal',
 'Work Comp Appeal',
 'Full Payment',
 'Full Payment',
 'No Discount',
 'Non-Par Provider',
 'Rate Update',
 'Non-Par Provider',
 'Red Flag and Appeal',
 'Non-Par Provider',
 'Negotiated Discount',
 'Full Payment',
 'Upfront Payment',
 'Discount Guideline']

In [37]:
eval(output)

['Non-Par Provider',
 'Payment Appeal',
 'Full Payment',
 'Red Flag',
 'Work Comp Appeal',
 'Work Comp Appeal',
 'Work Comp Appeal',
 'Work Comp Appeal',
 'Full Payment',
 'Full Payment',
 'No Discount',
 'Non-Par Provider',
 'Rate Update',
 'Non-Par Provider',
 'Red Flag and Appeal',
 'Non-Par Provider',
 'Negotiated Discount',
 'Upfront Payment',
 'Upfront Payment',
 'Discount Guideline']

In [35]:
eval(output)

['Non-Par Provider',
 'Payment Appeal',
 'Full Payment',
 'Red Flag',
 'Work Comp Appeal',
 'Work Comp Appeal',
 'Work Comp Appeal',
 'Work Comp Appeal',
 'Full Payment',
 'Full Payment',
 'No Discount',
 'Non-Par Provider',
 'Rate Update',
 'Non-Par Provider',
 'Red Flag and Appeal',
 'Non-Par Provider',
 'Negotiated Discount',
 'Upfront Payment',
 'Upfront Payment',
 'Discount Guideline']

In [36]:
list_of_notes[:20]

['8/23/24 - bk - Non Par for QON/Broadspire',
 'appealed pmt - 100% went to deduct',
 'process 100%',
 '6/18/24 - bk - Please red flag for Novanet.',
 'appeals CT work for FS after 4/1/2015',
 'appeals CT work comp',
 'appealed wc',
 'appeals CT work comp below FS',
 'provider balance bills, process 100%',
 'process in full',
 'provider does not accept discount, PROCESS at 100%',
 '12/22/24 - bk - Non par for Paradigm',
 '*Medben* - update rate for 86015',
 '11/19/24 - bk - Non par TIN.',
 '7/2/24 - bk - please red flag for Novanet.\r\n\r\nappeals MN wc, under 100 beds',
 '10/3/24 - bk - Non Par for Novanet',
 'standard negot discount of 10%',
 'Feder pays upfront - process at 100%',
 'member pays upfront',
 'they want rates closer to "BCBS of Alabama" which averages a 20% discount. So dont discount more than 20%']